In [ ]:
%pip install warcio

In [6]:
%pip install chardet

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [96]:
patterns = {
    "j.people.com.cn": { "start": r'<div class="txt_2 clearfix">', "end": r'<div class="page_n clearfix"></div>' }, 
    "nordot.app": { "start": r'<article class="ma">', "end": r'</article>' }, 
    "race.sanspo.com": { "start": r'<!--__pis_article_start__-->', "end": r'<!--__pis_article_end__-->' },
    "this.kiji.is": { "start": r'<article class="ma">', "end": r'</article>' },  
    "toyokeizai.net": { "start": r'<div id="article-body-inner"', "end": r'</div><!-- /article-body-inner -->' },  
    "www.chosunonline.com": { "start": r"chosunonline.com/js/ad_article_in.js'>", "end": r'</div><!--#article--><!-- google_ad_section_end -->' }, 
    "blogos.com": { "start": r'<div itemprop="articleBody" class="blogos_article">', "end": r'</div><!-- /blogos_article -->' },
    "digital.hakoshin.jp": { "start": r'<meta name="description" content="', "end": r'"> <meta name="keywords" content' },
    "europe.nna.jp": { "start": r'<div class="lead" itemprop="articleBody" >', "end": r'</div>' },  
    "hochi.news": { "start": r'<!-- cxenseparse_start -->', "end": r'<!-- cxenseparse_end -->' },  
    "jp.sputniknews.com": { "start": r'h2 itemprop="description" class="b-article__lead">', "end": r'</p></div><div class="social-likes-pane__holder">' }, 
    "news.livedoor.com": { "start": r'<span itemprop="articleBody">', "end": r'</span>' },  
    "response.jp": { "start": r'<!-- logly_body_begin --><p>', "end": r'</p><!-- logly_body_end -->' },
    "www.afpbb.com": { "start": r'<div class="article-body clear">', "end": r'<!-- cxenseparse_end -->' }, 
    "www.bloomberg.co.jp": { "start": r'<meta name="description" content="', "end": r'<meta http-equiv="X-UA-Compatible" content="IE=edge' },  
    "news.mynavi.jp": { "start": r'<div class="article-body" id="js-articleBody">', "end": r'<!-- cxenseparse_end -->' }, 
    "news.mynavi.jp-2016": { "start": r'<div itemprop="articleBody">', "end": r'<!-- #EndEditable -->' }
   

In [28]:
import re

def clean_text(text):
    # Убираем HTML-теги и URL
    clean_text = re.sub(r'<.*?>|http[s]?://S+', '', text)
    
    # Убираем лишние пробелы и символы переноса строки
    clean_text = re.sub(r'[\r\n\t]', ' ', clean_text)
    clean_text = re.sub(r's+', ' ', clean_text)

    return clean_text.strip()


In [8]:
import re
from warcio.archiveiterator import ArchiveIterator
from chardet.universaldetector import UniversalDetector
import os


def extract_text(file_path, patterns):
    collection = []

    detector = UniversalDetector()

    # Извлечение года из имени файла
    year = file_path[-15:-11]

    for domain, tags in patterns.items():
        if domain in file_path:
            start_tag = tags.get("start", "")
            end_tag = tags.get("end", "")

    with open(file_path, 'rb') as stream:
        for record in ArchiveIterator(stream):

            if record.rec_type == 'response':
                ext_text = record.raw_stream.read()
                detector.feed(ext_text)
                code = detector.result['encoding']
                if code is not None:
                    extr = ext_text.decode(code, errors='ignore')  # автоматическое определение кодировки
                else:
                    extr = ext_text.decode('utf-8', errors='ignore')
                match = re.search(f'{start_tag}(.*?){end_tag}', extr, re.DOTALL)
                if match:
                    text = match.group(1)
                    cleaned_text = clean_text(text)      # первичная очистка
                    record_uri = record.rec_headers.get_header('WARC-Target-URI')
                    collection.append(['ja', record_uri, year, cleaned_text])
    
    return collection


In [83]:
from pathlib import Path

# Путь к папке
folder_path = Path('/home/jupyter/datasphere/datasets/ja/ja')

# Получаем список всех файлов
files = [f for f in folder_path.iterdir() if f.is_file()]


In [100]:
import pandas as pd

# Создаем пустой DataFrame с указанными столбцами
columns = ['language', 'link', 'year', 'text']
empty_df = pd.DataFrame(columns=columns)

# Сохраняем DataFrame в CSV файл
output_path = '/home/jupyter/datasphere/project/ja_ready_data.csv'
empty_df.to_csv(output_path, index=False)

print(f"Пустой DataFrame сохранен в {output_path}")


Пустой DataFrame сохранен в /home/jupyter/datasphere/project/ja_ready_data.csv


In [ ]:
csv_file_path = '/home/jupyter/datasphere/project/ja_ready_data.csv'
for file in files:
    file_key = file.name.split('-')[0]
    if file_key in patterns:
        new_data_df = pd.DataFrame(extract_text('/home/jupyter/datasphere/datasets/ja/ja/'+file.name, patterns), columns = ['language', 'link', 'year', 'text'])

        # Чтение существующего CSV-файла
        existing_data_df = pd.read_csv(csv_file_path)

        # Объединение существующих и новых данных
        updated_data_df = pd.concat([existing_data_df, new_data_df], axis=0, ignore_index=True)

        # Запись обновленных данных обратно в CSV-файл
        updated_data_df.to_csv(csv_file_path, index=False)

In [ ]:
# ['ja',
#   'http://www.chosunonline.com/site/data/html_dir/2022/06/01/2022060180001.html',
#   '2022',
#   '【ソウル聯合ニュース】韓国の統一地方選と国会議員補欠選の投票が１日午前６時、и остальной текст']